# Architektur Neuronales Netz

In [1]:
# aNN Architektur

# Importe / Bibliotheken
import torch
from torch import nn
from torch import log10
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR, MultiStepLR, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


#### Default Datentyp

In [2]:
torch.set_default_dtype(torch.float64)

#### Erzeugnung des Moduls

In [3]:
class NeuralNetwork(nn.Module):
    
    #Initalisierung der Netzwerk layers
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
    
        super().__init__() #Referenz zur Base Class (nn.Module)
        #Kaskade der Layer
        self.linear_afunc_stack = nn.Sequential(
            nn.BatchNorm1d(input_size), # Normalisierung, damit Inputdaten gleiche Größenordnung haben
            nn.Linear(input_size, hidden1_size), #Lineare Transformation mit gespeicherten weights und biases
            #nn.LayerNorm(hidden1_size),
            nn.Tanh(), #Nicht lineare Aktivierungsfunktion um komplexe nichtlineare Zusammenhänge abzubilden
            #nn.SELU(),
            nn.Linear(hidden1_size, hidden2_size),
            #nn.LayerNorm(hidden2_size),
            nn.Tanh(),
            #nn.SELU(),
            nn.Linear(hidden2_size, hidden3_size),
            #nn.LayerNorm(hidden3_size),
            nn.Tanh(),
            #nn.SELU(),
            nn.Linear(hidden3_size, output_size),
        )

    #Implementierung der Operationen auf Input Daten
    def forward(self, x):
        out = self.linear_afunc_stack(x)
        return out

#### Ausgabe Modul

In [4]:
model = NeuralNetwork(5, 200, 200, 200, 1)
print(model)

NeuralNetwork(
  (linear_afunc_stack): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=200, bias=True)
    (2): Tanh()
    (3): Linear(in_features=200, out_features=200, bias=True)
    (4): Tanh()
    (5): Linear(in_features=200, out_features=200, bias=True)
    (6): Tanh()
    (7): Linear(in_features=200, out_features=1, bias=True)
  )
)


#### DataLoader

In [5]:
batch_size = 32 #Zahl der Datenpaare die vor einem erneuten Update der Parameter ins Netzt gegeben werden
eq_data_file = Path.cwd() / 'data' / 'eq_dataset.npz' #Import der GGW Daten

res = np.load(eq_data_file)

# Bei Speicherung wurden Daten als T, p, x_0 und xi gespeichert
# Inputs T, p, x_0[H2,N2,NH3]
# Outputs xi
# Umwandlen der np.arrays in torch.tensors zur besseren Arbeit mit PyTorch
T = torch.tensor(res['T'])
p = torch.tensor(res['p'])
x_0 = torch.tensor(res['x_0'])
xi = torch.tensor(res['xi'])

#Anpassen der Daten auf gleiche Größenordnung
#T = log10(T)
# T = T / 850
# p = p / 1000

# T = torch.tensor(res['T']).float()
# p = torch.tensor(res['p']).float()
# x_0 = torch.tensor(res['x_0']).float()
# xi = torch.tensor(res['xi']).float()

# print(T.dtype)
# print(xi.dtype)

x_input = torch.stack((T, p ,x_0[:,0],x_0[:,1],x_0[:,2]),1)
y_output = xi.reshape((-1,1))
#print(x_input.size())
# print(xi.size())

# Tensoren zu einem großen Set gruppieren
dataset = TensorDataset(x_input, y_output)

# for x,y in dataset:
#     print(x)
#     print(y)
    
# Split in Trainings und Test Set
train_dataset, test_dataset = random_split(dataset, 
                                           [int(0.8*len(dataset)), int(0.2*len(dataset))], # splitting 80/20
                                           generator = torch.Generator().manual_seed(42)) # Festlegung seed zur Reproduktivität

# Erzeugen der DataLoader zur Arbeit mit Daten
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True) # shuffle batches zur Reduzierung von overfitting
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)


#### Generierung Netzwerk, Festlegung von loss Funktion und Optimizer

In [6]:
# Erzeugung aNN
net = NeuralNetwork(5, 200, 200, 200, 1)

# Loss Funktion; gibt Fehler an
#loss_fn = nn.MSELoss()
loss_fn = nn.L1Loss()

#Definition custom loss Funktion, MRE
def MRELoss(outputs, targets):
    
    loss = torch.mean(abs((outputs - targets) / targets))
    
    return loss
    

#Optimizer
learning_rate = 1e-2
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)
#scheduler = StepLR(optimizer, step_size = 30, gamma = 0.1)
#scheduler = MultiStepLR(optimizer, milestones=[30, 70, 100], gamma = 0.1)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 10, threshold = 1e-5)

#### Funktion zur Bestimmung der Genauigkeit

In [7]:
def check_accuracy(loader, net):
    
    loss = 0
    MRE = 0
    train_correct = 0
    train_total = len(loader.dataset)
    num_batches = len(loader) 
    #train_total = 0
    
    net.eval() # Put network in evaluation mode
    
    if loader == train_dataloader:
        dataset = "Train"
    else:
        dataset = "Test"
    
    with torch.no_grad():
        for X, y in loader:
            pred = net(X)
           
            #loss += MRELoss(pred, y).item()
            loss += loss_fn(pred, y) # Calculate the loss
            MRE += MRELoss(pred, y)

            # Record the correct predictions for training data
            #_, predictions = torch.max(pred.data, 1)
            for i in range(len(pred)):
                if pred[i] - y[i] <= 0.01:
                    train_correct += 1
            #train_correct += (abs(pred.argmax(1) - y) <= 0.01).sum().item()
            #train_correct += (abs(predictions - y.data) <= 0.01).sum()
            #train_total += predictions.size(0)
            
        # Genauigkeit berechnen
        acc = float(train_correct) / float(train_total) * 100
        acc = round(acc, 2)
        
        loss /= num_batches
        MRE /= num_batches
        print(f"{dataset} Error: \n Accuracy: {acc}%, Avg loss: {loss:>8f}, MRE: {MRE:>8f} \n")

    net.train()
    
    return acc, loss, MRE

#### Training

In [9]:
num_epochs = 200 #Iterationen über Datenset

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
train_MRE = []
test_MRE = []

#Optimierungsloop
for epoch in range(num_epochs):
#     train_correct = 0
#     train_total = 0
        
    for batch, (X,y) in enumerate(train_dataloader):
        
#         print(X.shape)
#         print(X.dtype)
        
        net.train() #Trainingmodus
        
        # forward
        pred = net(X)  # Do the forward pass
        loss = loss_fn(pred, y) # Calculate the loss
        #loss = MRELoss(pred, y)
        
        # backward
        optimizer.zero_grad() # Clear off the gradients from any past operation
        loss.backward()       # Calculate the gradients with help of back propagation, updating weights and biases
        
        # adam step gradient descent
        optimizer.step()      # Ask the optimizer to adjust the parameters based on the gradients

#         # Record the correct predictions for training data
#         _, predictions = torch.max(pred.data, 1)
#         train_correct += (predictions == y.data).sum()                
#         train_total += predictions.size(0)    

        print ('Epoch %d/%d, Iteration %d/%d, Loss: %.4f' 
               %(epoch+1, num_epochs, batch+1, len(train_dataset)//batch_size, loss.item()))
        
    
    #scheduler.step() # Reduzieren Learning Rate (falls step size erreicht)
    net.eval() # Put the network into evaluation mode
    
    # Book keeping
    #train_loss.append(loss.item())
    
    # What was our train accuracy?
    tr_acc, tr_loss, tr_MRE = check_accuracy(train_dataloader, net)
    
    #Record loss and accuracy
    train_accuracy.append(tr_acc)
    train_loss.append(tr_loss)
    train_MRE.append(tr_MRE)
    
    scheduler.step(tr_loss) # Aufruf LR scheduler für reduce on plateau
    
    # How did we do on the test set (the unseen set)
    # Record the correct predictions for test data
    t_acc, t_loss, t_MRE = check_accuracy(test_dataloader, net)
    test_accuracy.append(t_acc)
    test_loss.append(t_loss)
    test_MRE.append(t_MRE)

#     loss = criterion(outputs, Variable(test_classes))
#     test_loss.append(loss.data[0])

Epoch 1/200, Iteration 1/25, Loss: 0.1516
Epoch 1/200, Iteration 2/25, Loss: 0.2117
Epoch 1/200, Iteration 3/25, Loss: 0.1546
Epoch 1/200, Iteration 4/25, Loss: 0.0597
Epoch 1/200, Iteration 5/25, Loss: 0.1436
Epoch 1/200, Iteration 6/25, Loss: 0.2316
Epoch 1/200, Iteration 7/25, Loss: 0.1090
Epoch 1/200, Iteration 8/25, Loss: 0.0903
Epoch 1/200, Iteration 9/25, Loss: 0.1488
Epoch 1/200, Iteration 10/25, Loss: 0.1036
Epoch 1/200, Iteration 11/25, Loss: 0.0972
Epoch 1/200, Iteration 12/25, Loss: 0.0756
Epoch 1/200, Iteration 13/25, Loss: 0.1159
Epoch 1/200, Iteration 14/25, Loss: 0.0915
Epoch 1/200, Iteration 15/25, Loss: 0.0831
Epoch 1/200, Iteration 16/25, Loss: 0.0715
Epoch 1/200, Iteration 17/25, Loss: 0.0474
Epoch 1/200, Iteration 18/25, Loss: 0.0857
Epoch 1/200, Iteration 19/25, Loss: 0.0788
Epoch 1/200, Iteration 20/25, Loss: 0.0547
Epoch 1/200, Iteration 21/25, Loss: 0.1878
Epoch 1/200, Iteration 22/25, Loss: 0.1383
Epoch 1/200, Iteration 23/25, Loss: 0.0729
Epoch 1/200, Iterati

Epoch 8/200, Iteration 19/25, Loss: 0.0538
Epoch 8/200, Iteration 20/25, Loss: 0.0578
Epoch 8/200, Iteration 21/25, Loss: 0.0571
Epoch 8/200, Iteration 22/25, Loss: 0.0589
Epoch 8/200, Iteration 23/25, Loss: 0.0648
Epoch 8/200, Iteration 24/25, Loss: 0.0567
Epoch 8/200, Iteration 25/25, Loss: 0.0393
Train Error: 
 Accuracy: 52.5%, Avg loss: 0.051780, MRE: 3.397577 

Test Error: 
 Accuracy: 52.0%, Avg loss: 0.058185, MRE: 9.153481 

Epoch 9/200, Iteration 1/25, Loss: 0.0489
Epoch 9/200, Iteration 2/25, Loss: 0.0480
Epoch 9/200, Iteration 3/25, Loss: 0.0729
Epoch 9/200, Iteration 4/25, Loss: 0.0684
Epoch 9/200, Iteration 5/25, Loss: 0.0332
Epoch 9/200, Iteration 6/25, Loss: 0.0360
Epoch 9/200, Iteration 7/25, Loss: 0.0440
Epoch 9/200, Iteration 8/25, Loss: 0.0454
Epoch 9/200, Iteration 9/25, Loss: 0.0317
Epoch 9/200, Iteration 10/25, Loss: 0.0446
Epoch 9/200, Iteration 11/25, Loss: 0.0633
Epoch 9/200, Iteration 12/25, Loss: 0.0668
Epoch 9/200, Iteration 13/25, Loss: 0.0503
Epoch 9/200, I

Epoch 16/200, Iteration 12/25, Loss: 0.0568
Epoch 16/200, Iteration 13/25, Loss: 0.0452
Epoch 16/200, Iteration 14/25, Loss: 0.0664
Epoch 16/200, Iteration 15/25, Loss: 0.0896
Epoch 16/200, Iteration 16/25, Loss: 0.0368
Epoch 16/200, Iteration 17/25, Loss: 0.0674
Epoch 16/200, Iteration 18/25, Loss: 0.0492
Epoch 16/200, Iteration 19/25, Loss: 0.0723
Epoch 16/200, Iteration 20/25, Loss: 0.0490
Epoch 16/200, Iteration 21/25, Loss: 0.0746
Epoch 16/200, Iteration 22/25, Loss: 0.0565
Epoch 16/200, Iteration 23/25, Loss: 0.1519
Epoch 16/200, Iteration 24/25, Loss: 0.1342
Epoch 16/200, Iteration 25/25, Loss: 0.0598
Train Error: 
 Accuracy: 53.0%, Avg loss: 0.130751, MRE: 6.187902 

Test Error: 
 Accuracy: 52.5%, Avg loss: 0.141669, MRE: 15.108296 

Epoch 17/200, Iteration 1/25, Loss: 0.1409
Epoch 17/200, Iteration 2/25, Loss: 0.0920
Epoch 17/200, Iteration 3/25, Loss: 0.0852
Epoch 17/200, Iteration 4/25, Loss: 0.1283
Epoch 17/200, Iteration 5/25, Loss: 0.0733
Epoch 17/200, Iteration 6/25, Los

Epoch 23/200, Iteration 4/25, Loss: 0.1292
Epoch 23/200, Iteration 5/25, Loss: 0.0380
Epoch 23/200, Iteration 6/25, Loss: 0.1145
Epoch 23/200, Iteration 7/25, Loss: 0.1605
Epoch 23/200, Iteration 8/25, Loss: 0.1741
Epoch 23/200, Iteration 9/25, Loss: 0.0840
Epoch 23/200, Iteration 10/25, Loss: 0.0707
Epoch 23/200, Iteration 11/25, Loss: 0.1347
Epoch 23/200, Iteration 12/25, Loss: 0.1285
Epoch 23/200, Iteration 13/25, Loss: 0.0735
Epoch 23/200, Iteration 14/25, Loss: 0.1479
Epoch 23/200, Iteration 15/25, Loss: 0.0960
Epoch 23/200, Iteration 16/25, Loss: 0.0751
Epoch 23/200, Iteration 17/25, Loss: 0.1425
Epoch 23/200, Iteration 18/25, Loss: 0.1460
Epoch 23/200, Iteration 19/25, Loss: 0.1128
Epoch 23/200, Iteration 20/25, Loss: 0.1808
Epoch 23/200, Iteration 21/25, Loss: 0.1228
Epoch 23/200, Iteration 22/25, Loss: 0.1189
Epoch 23/200, Iteration 23/25, Loss: 0.1526
Epoch 23/200, Iteration 24/25, Loss: 0.1389
Epoch 23/200, Iteration 25/25, Loss: 0.1095
Train Error: 
 Accuracy: 59.0%, Avg lo

Train Error: 
 Accuracy: 74.25%, Avg loss: 0.019879, MRE: 1.131251 

Test Error: 
 Accuracy: 71.0%, Avg loss: 0.021701, MRE: 4.291892 

Epoch 30/200, Iteration 1/25, Loss: 0.0179
Epoch 30/200, Iteration 2/25, Loss: 0.0250
Epoch 30/200, Iteration 3/25, Loss: 0.0256
Epoch 30/200, Iteration 4/25, Loss: 0.0267
Epoch 30/200, Iteration 5/25, Loss: 0.0199
Epoch 30/200, Iteration 6/25, Loss: 0.0227
Epoch 30/200, Iteration 7/25, Loss: 0.0258
Epoch 30/200, Iteration 8/25, Loss: 0.0278
Epoch 30/200, Iteration 9/25, Loss: 0.0224
Epoch 30/200, Iteration 10/25, Loss: 0.0263
Epoch 30/200, Iteration 11/25, Loss: 0.0282
Epoch 30/200, Iteration 12/25, Loss: 0.0311
Epoch 30/200, Iteration 13/25, Loss: 0.0312
Epoch 30/200, Iteration 14/25, Loss: 0.0245
Epoch 30/200, Iteration 15/25, Loss: 0.0221
Epoch 30/200, Iteration 16/25, Loss: 0.0256
Epoch 30/200, Iteration 17/25, Loss: 0.0226
Epoch 30/200, Iteration 18/25, Loss: 0.0182
Epoch 30/200, Iteration 19/25, Loss: 0.0269
Epoch 30/200, Iteration 20/25, Loss: 

...

Train Error: 
 Accuracy: 94.25%, Avg loss: 0.005087, MRE: 0.282330 

Test Error: 
 Accuracy: 93.5%, Avg loss: 0.006410, MRE: 0.690572 

Epoch 167/200, Iteration 1/25, Loss: 0.0059
Epoch 167/200, Iteration 2/25, Loss: 0.0070
Epoch 167/200, Iteration 3/25, Loss: 0.0116
Epoch 167/200, Iteration 4/25, Loss: 0.0086
Epoch 167/200, Iteration 5/25, Loss: 0.0061
Epoch 167/200, Iteration 6/25, Loss: 0.0113
Epoch 167/200, Iteration 7/25, Loss: 0.0338
Epoch 167/200, Iteration 8/25, Loss: 0.0060
Epoch 167/200, Iteration 9/25, Loss: 0.0074
Epoch 167/200, Iteration 10/25, Loss: 0.0078
Epoch 167/200, Iteration 11/25, Loss: 0.0247
Epoch 167/200, Iteration 12/25, Loss: 0.0157
Epoch 167/200, Iteration 13/25, Loss: 0.0102
Epoch 167/200, Iteration 14/25, Loss: 0.0104
Epoch 167/200, Iteration 15/25, Loss: 0.0089
Epoch 167/200, Iteration 16/25, Loss: 0.0092
Epoch 167/200, Iteration 17/25, Loss: 0.0119
Epoch 167/200, Iteration 18/25, Loss: 0.0132
Epoch 167/200, Iteration 19/25, Loss: 0.0089
Epoch 167/200, Ite

Train Error: 
 Accuracy: 96.25%, Avg loss: 0.005004, MRE: 0.252633 

Test Error: 
 Accuracy: 95.0%, Avg loss: 0.006350, MRE: 0.938580 

Epoch 174/200, Iteration 1/25, Loss: 0.0153
Epoch 174/200, Iteration 2/25, Loss: 0.0223
Epoch 174/200, Iteration 3/25, Loss: 0.0203
Epoch 174/200, Iteration 4/25, Loss: 0.0117
Epoch 174/200, Iteration 5/25, Loss: 0.0076
Epoch 174/200, Iteration 6/25, Loss: 0.0158
Epoch 174/200, Iteration 7/25, Loss: 0.0093
Epoch 174/200, Iteration 8/25, Loss: 0.0141
Epoch 174/200, Iteration 9/25, Loss: 0.0065
Epoch 174/200, Iteration 10/25, Loss: 0.0261
Epoch 174/200, Iteration 11/25, Loss: 0.0048
Epoch 174/200, Iteration 12/25, Loss: 0.0141
Epoch 174/200, Iteration 13/25, Loss: 0.0121
Epoch 174/200, Iteration 14/25, Loss: 0.0124
Epoch 174/200, Iteration 15/25, Loss: 0.0098
Epoch 174/200, Iteration 16/25, Loss: 0.0062
Epoch 174/200, Iteration 17/25, Loss: 0.0100
Epoch 174/200, Iteration 18/25, Loss: 0.0287
Epoch 174/200, Iteration 19/25, Loss: 0.0069
Epoch 174/200, Ite

Train Error: 
 Accuracy: 94.5%, Avg loss: 0.004969, MRE: 0.272596 

Test Error: 
 Accuracy: 94.0%, Avg loss: 0.006225, MRE: 0.848866 

Epoch 181/200, Iteration 1/25, Loss: 0.0117
Epoch 181/200, Iteration 2/25, Loss: 0.0351
Epoch 181/200, Iteration 3/25, Loss: 0.0058
Epoch 181/200, Iteration 4/25, Loss: 0.0123
Epoch 181/200, Iteration 5/25, Loss: 0.0082
Epoch 181/200, Iteration 6/25, Loss: 0.0171
Epoch 181/200, Iteration 7/25, Loss: 0.0106
Epoch 181/200, Iteration 8/25, Loss: 0.0152
Epoch 181/200, Iteration 9/25, Loss: 0.0238
Epoch 181/200, Iteration 10/25, Loss: 0.0251
Epoch 181/200, Iteration 11/25, Loss: 0.0108
Epoch 181/200, Iteration 12/25, Loss: 0.0158
Epoch 181/200, Iteration 13/25, Loss: 0.0209
Epoch 181/200, Iteration 14/25, Loss: 0.0081
Epoch 181/200, Iteration 15/25, Loss: 0.0083
Epoch 181/200, Iteration 16/25, Loss: 0.0185
Epoch 181/200, Iteration 17/25, Loss: 0.0126
Epoch 181/200, Iteration 18/25, Loss: 0.0086
Epoch 181/200, Iteration 19/25, Loss: 0.0166
Epoch 181/200, Iter

Epoch 187/200, Iteration 21/25, Loss: 0.0093
Epoch 187/200, Iteration 22/25, Loss: 0.0075
Epoch 187/200, Iteration 23/25, Loss: 0.0214
Epoch 187/200, Iteration 24/25, Loss: 0.0088
Epoch 187/200, Iteration 25/25, Loss: 0.0116
Train Error: 
 Accuracy: 97.88%, Avg loss: 0.005251, MRE: 0.243255 

Test Error: 
 Accuracy: 96.0%, Avg loss: 0.006857, MRE: 1.135648 

Epoch 188/200, Iteration 1/25, Loss: 0.0081
Epoch 188/200, Iteration 2/25, Loss: 0.0097
Epoch 188/200, Iteration 3/25, Loss: 0.0157
Epoch 188/200, Iteration 4/25, Loss: 0.0118
Epoch 188/200, Iteration 5/25, Loss: 0.0196
Epoch 188/200, Iteration 6/25, Loss: 0.0125
Epoch 188/200, Iteration 7/25, Loss: 0.0132
Epoch 188/200, Iteration 8/25, Loss: 0.0126
Epoch 188/200, Iteration 9/25, Loss: 0.0102
Epoch 188/200, Iteration 10/25, Loss: 0.0132
Epoch 188/200, Iteration 11/25, Loss: 0.0147
Epoch 188/200, Iteration 12/25, Loss: 0.0228
Epoch 188/200, Iteration 13/25, Loss: 0.0113
Epoch 188/200, Iteration 14/25, Loss: 0.0136
Epoch 188/200, Ite

Train Error: 
 Accuracy: 87.25%, Avg loss: 0.005789, MRE: 0.332669 

Test Error: 
 Accuracy: 84.0%, Avg loss: 0.006607, MRE: 0.730337 

Epoch 195/200, Iteration 1/25, Loss: 0.0161
Epoch 195/200, Iteration 2/25, Loss: 0.0071
Epoch 195/200, Iteration 3/25, Loss: 0.0116
Epoch 195/200, Iteration 4/25, Loss: 0.0078
Epoch 195/200, Iteration 5/25, Loss: 0.0182
Epoch 195/200, Iteration 6/25, Loss: 0.0079
Epoch 195/200, Iteration 7/25, Loss: 0.0089
Epoch 195/200, Iteration 8/25, Loss: 0.0122
Epoch 195/200, Iteration 9/25, Loss: 0.0120
Epoch 195/200, Iteration 10/25, Loss: 0.0246
Epoch 195/200, Iteration 11/25, Loss: 0.0055
Epoch 195/200, Iteration 12/25, Loss: 0.0130
Epoch 195/200, Iteration 13/25, Loss: 0.0108
Epoch 195/200, Iteration 14/25, Loss: 0.0200
Epoch 195/200, Iteration 15/25, Loss: 0.0175
Epoch 195/200, Iteration 16/25, Loss: 0.0382
Epoch 195/200, Iteration 17/25, Loss: 0.0074
Epoch 195/200, Iteration 18/25, Loss: 0.0247
Epoch 195/200, Iteration 19/25, Loss: 0.0090
Epoch 195/200, Ite

#### Plots loss vs Epoch

In [ ]:
fig,ax = plt.subplots()
#fig.figsize=(12, 8)
ax.semilogy(train_loss, label='train loss')
ax.semilogy(test_loss, label='test loss')
plt.title("Train and Test Loss")
ax.set(xlabel = '$Epochs$ / 1', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol') #Beschriftung Achsen; Kursiv durch $$; Index durch _{}
ax.tick_params(direction = 'in') #, length = 20, width = 3)

plt.legend()
plt.tight_layout()
plt.show()

#### Parity Plot

In [ ]:
xi_real = []
xi_pred = []
for (X,y) in train_dataloader:
    xi_real = np.append(xi_real, y.numpy())
    xi_pred = np.append(xi_pred, net(X).detach().numpy())

xi_real_test = []
xi_pred_test = []
for (X,y) in test_dataloader:
    xi_real_test = np.append(xi_real_test, y.numpy())
    xi_pred_test = np.append(xi_pred_test, net(X).detach().numpy())

# find the boundaries of X and Y values
bounds = (min(xi_real.min(), xi_pred.min()) - int(0.1 * xi_pred.min()), max(xi_real.max(), xi_pred.max())+ int(0.1 * xi_pred.max()))

fig,ax = plt.subplots(1,2, figsize =(10,10))

# # Reset the limits
# ax[0] = plt.gca()
ax[0].set_xlim(bounds)
ax[0].set_ylim(bounds)
# Ensure the aspect ratio is square
ax[0].set_aspect("equal", adjustable="box")

ax[0].plot(xi_real, xi_pred, '.')
ax[0].plot([0, 1], [0, 1], "r-",lw=2 ,transform=ax[0].transAxes)
ax[0].set(xlabel = '$\\xi$ / mol', ylabel = '$\\xi\mathregular{_{pred}}$ / mol')
ax[0].tick_params(direction = 'in') #, length = 20, width = 3)
ax[0].set_title('Train Data')

# Reset the limits
#ax[1] = plt.gca()
ax[1].set_xlim(bounds)
ax[1].set_ylim(bounds)
# Ensure the aspect ratio is square
ax[1].set_aspect("equal", adjustable="box")

ax[1].plot(xi_real_test, xi_pred_test, '.')
ax[1].plot([0, 1], [0, 1], "r-",lw=2 ,transform=ax[1].transAxes)
ax[1].set(xlabel = '$\\xi$ / mol', ylabel = '$\\xi\mathregular{_{pred}}$ / mol')
ax[1].tick_params(direction = 'in') #, length = 20, width = 3)
ax[1].set_title('Test Data')


#plt.legend()
#fig.suptitle("Parity Plot")
plt.tight_layout()
plt.show()

#### Plot Fehler vs Epoch

In [ ]:
fig,ax = plt.subplots()
#fig.figsize=(12, 8)
ax.plot(train_MRE, label='train MRE')
ax.plot(test_MRE, label='test MRE')
plt.title("Mean Relative Error")
ax.set(xlabel = '$Epochs$ / 1', ylabel = '$MRE$ / mol') #Beschriftung Achsen; Kursiv durch $$; Index durch _{}
ax.tick_params(direction = 'in') #, length = 20, width = 3)

plt.legend()
plt.tight_layout()
plt.show()

#### Plot Loss vs Variable Properties

In [ ]:
mistake = []
param_T = []
param_p = []
param_x_H2 = []
param_x_N2 = []
param_x_NH3 = []
for X,y in train_dataloader:
    mistake = np.append(mistake, abs(y - net(X).detach().numpy()))
    param_T = np.append(param_T, X[:,0])
    param_p = np.append(param_p, X[:,1])
    param_x_H2 = np.append(param_x_H2, X[:,2])
    param_x_N2 = np.append(param_x_N2, X[:,3])
    param_x_NH3 = np.append(param_x_NH3, X[:,4])
    
# train_parameters, train_xi = next(iter(train_dataloader))
# y = abs(train_xi - net(train_parameters).detach().numpy())
# #[T, p ,x_H2, x_N2, x_NH3]
# x = [train_parameters[:,0], train_parameters[:,1], train_parameters[:,2], train_parameters[:,3], train_parameters[:,4]]

# print(param_T[0])
# print(param_T)
# print(mistake)

fig,ax = plt.subplots(1,3, figsize = (10, 5), gridspec_kw={'width_ratios': [2,2,3]})

ax[0].plot(param_T, mistake, '.', markersize = 2)
ax[0].set(xlabel = '$T$ / K', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol')
ax[0].tick_params(direction = 'in') #, length = 20, width = 3)

ax[1].plot(param_p, mistake, '.', markersize = 2)
ax[1].set(xlabel = '$p$ / bar', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol')
ax[1].tick_params(direction = 'in') #, length = 20, width = 3)

ax[2].plot(param_x_H2, mistake, '.', markersize = 2, label = '$x\mathregular{_{H_2}}$')
ax[2].plot(param_x_N2, mistake, '.', markersize = 2, label = '$x\mathregular{_{N_2}}$')
ax[2].plot(param_x_NH3, mistake, '.', markersize = 2, label = '$x\mathregular{_{NH_3}}$')
ax[2].set(xlabel = '$x\mathregular{_{NH_3}}$ / bar', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol')
ax[2].tick_params(direction = 'in') #, length = 20, width = 3)
ax[2].set

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots()
#fig.figsize=(12, 8)
ax.plot(train_accuracy, label='train accuracy')
ax.plot(test_accuracy, label='test accuracy')
plt.title("Train and Test Accuracy")
ax.set(xlabel = '$Epochs$ / 1', ylabel = '$Accuracy$ / %') #Beschriftung Achsen; Kursiv durch $$; Index durch _{}
ax.tick_params(direction = 'in') #, length = 20, width = 3)

plt.legend()
plt.tight_layout()
plt.show()

#### Debugging Hilfe

In [ ]:
# Anzeigen aller Input X und Output y Daten
for (X,y) in train_dataloader:
    print(X)
    print(y.reshape((-1,1)))
    print(net(X))
    break

#### Einblick in Netzwerk

In [ ]:
list(net.parameters()) # zeigt weights, biases, ...

In [ ]:
X = torch.rand((2,5))
print(X)
print(net(X))

In [ ]:
Lnorm = nn.LayerNorm(5)
Bnorm = nn.BatchNorm1d(5)

In [ ]:
for (X,y) in train_dataloader:
    print(X)
    #print(y.reshape((-1,1)))
    print(Bnorm(X).mean(dim=0))
    print(Bnorm(X))
    break